<a href="https://colab.research.google.com/github/rajy4683/EIP4_Phase1_Final/blob/master/OCP_DavNetSep_A5_wrn_acc_gml_wrn_ddd_sth_rjy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')

!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 



Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dense, Conv2D, BatchNormalization, Activation

#import keras_one_cycle_clr as ktool


Using TensorFlow backend.


In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.callbacks import CyclicLR

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-43c64pya
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-43c64pya
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=51b1f2dee5142d335ebb6db1dd6ef01560ebd32feb484f4ae33bf20704437940
  Stored in directory: /tmp/pip-ephem-wheel-cache-vdclh4ho/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import tensorflow as tf

from tqdm import tqdm_notebook as tqdm

import tensorflow.contrib.eager as tfe
#tf.enable_eager_execution()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
from keras_contrib.callbacks import CyclicLR


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
one_hot_df.head()


,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
0,resized/1.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
1,resized/2.jpg,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0
2,resized/3.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,resized/4.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,resized/5.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0


In [0]:
import keras
import numpy as np
from google.colab.patches import cv2_imshow
from tensorflow.python.keras.utils.data_utils import Sequence

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]



#class PersonDataGenerator(keras.utils.Sequence):
class PersonDataGenerator(Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,normalize=False,aug_flow=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.normalize = normalize
        self.aug_flow=aug_flow
        self.on_epoch_end()
        #print("Shuffle = ",self.shuffle)

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        #print(batch_slice)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        #print(items["image_path"])
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }


        if(self.aug_flow is not None):
            image = self.aug_flow.flow(image,shuffle=False,batch_size=self.batch_size).next()

        if(self.normalize == True):
            train_mean = np.mean(image, axis=(0,1,2))
            train_std = np.std(image, axis=(0,1,2))
            #print(train_mean, train_std)
            normalize = lambda x: ((x - train_mean) / train_std).astype('float32')
            image = normalize(image)

        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_old_df = pd.read_csv('/content/gdrive/My Drive/DavNet_Base/train_df_davnetsep_widrn_gml2_1577343325.csv')
train_old_df.shape
train_df = train_old_df
val_old_df = pd.read_csv('/content/gdrive/My Drive/DavNet_Base/val_df_davnetsep_widrn_gml2_1577343325.csv')
val_old_df.shape
val_df = val_old_df
print(train_df.shape, val_df.shape)

(11537, 28) (2036, 28)


In [0]:
from datetime import datetime
def get_curr_time():
    return int(datetime.utcnow().strftime("%s"))

model_name_itr = 'davnetsep_widrn_gml2_wrn_ddd_sth_rjy_'+str(get_curr_time())
gdrive_home_path="/content/gdrive/My Drive/DavNet_Base/"
train_csv=gdrive_home_path+"train_df_"+model_name_itr+".csv"
val_csv=gdrive_home_path+"val_df_"+model_name_itr+".csv"
json_file=gdrive_home_path+"json_"+model_name_itr+".json"
png_file=gdrive_home_path+"png_"+model_name_itr+".png"
weights_file=gdrive_home_path+"h5_"+model_name_itr+".h5"

print("Model-name:",model_name_itr)
print(train_csv,val_csv,json_file,png_file,weights_file)
train_df.to_csv(train_csv, index=False)
val_df.to_csv(val_csv, index=False)




Model-name: davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612
/content/gdrive/My Drive/DavNet_Base/train_df_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.csv /content/gdrive/My Drive/DavNet_Base/val_df_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.csv /content/gdrive/My Drive/DavNet_Base/json_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.json /content/gdrive/My Drive/DavNet_Base/png_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.png /content/gdrive/My Drive/DavNet_Base/h5_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.h5


In [0]:
print("Model-name:",model_name_itr)
for var_name in [train_csv,val_csv,json_file,png_file,weights_file]:
  print(var_name)

Model-name: davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612
/content/gdrive/My Drive/DavNet_Base/train_df_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.csv
/content/gdrive/My Drive/DavNet_Base/val_df_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.csv
/content/gdrive/My Drive/DavNet_Base/json_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.json
/content/gdrive/My Drive/DavNet_Base/png_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.png
/content/gdrive/My Drive/DavNet_Base/h5_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.h5


In [0]:
_gender_cols_

In [0]:
colname="age_15-25"

In [0]:
def equalize_classwise_dist(df_to_equalize, selector_column, classwise_dist):
    max_value = np.max(classwise_dist)
    max_column = selector_column[np.argmax(classwise_dist)]
    equalizer_const = [ np.floor((max_value-value)/value).astype('int') for value in classwise_dist]
    print(equalizer_const)
    print(equalizer_const[0])
    for counter in range(len(equalizer_const)):
        local_df = pd.DataFrame()
        column_to_select = selector_column[counter]
        for value in range(equalizer_const[counter]):
            #,equalizer_const[counter]) # The actual class
            #print(df_to_equalize[df_to_equalize[column_to_select]==1].shape[0])
            local_df = local_df.append(df_to_equalize[df_to_equalize[column_to_select]==1])

        if (local_df.empty == False):
            print("Appending ",local_df.shape[0]," rows for ",selector_column[counter])
            print("Current count for ",selector_column[counter], ":",df_to_equalize[df_to_equalize[column_to_select]==1].shape[0])
            #df_to_equalize = df_to_equalize.append(local_df, ignore_index=True)
            #df_to_equalize.reset_index()
            #df_to_equalize=pd.concat([df_to_equalize,local_df], axis=0,ignore_index=True)

            print("New count for ",selector_column[counter], ":",df_to_equalize[df_to_equalize[column_to_select]==1].shape[0])
    print(local_df.shape)
    return local_df


In [0]:
output_weights = ["gender_output", "imagequality_ouput","age_output", "weight_output", "bag_output", "footwear_output","emotion_output", "pose_output"]
col_splits = [_gender_cols_, _imagequality_cols_,_age_cols_, _weight_cols_, _carryingbag_cols_, _footwear_cols_, _emotion_cols_, _bodypose_cols_]
def get_dist(train_df,equalize_classes=False):
    loss_weights = {}
    index=0
    for selector_column in col_splits:
        print(selector_column)
        count = []
        percentile = []
        for age_split in selector_column:
            count.append( train_df[selector_column][train_df[age_split] == 1].shape[0])
        #print(count, np.round((count/11537.0)*100.0, 2))

        max_val = np.max(count)
        total_count = np.float32(train_df.shape[0])
        #print(count, )
        count_weights= [np.round(max_val/current_val,3) for current_val in count]
        print(count_weights)
        #np.round((np.asarray(count)/total)*100.0, 2)

        #print("Top Class:",selector_column[np.argmax(count)],"Max Count",np.max(count))
        #print("Bottom Class:",selector_column[np.argmin(count)])
        #weights_dist = dict(zip(selector_column, count_weights))
        
        #print(weights_dist)
        weights_vals_dist={}
        index_val=0
        for y in range(len(count_weights)):
            weights_vals_dist[y]=count_weights[y]
            print(weights_vals_dist[y],y)
            #index_val+=index_val
            
            #loss_weights[output_weights[index]]={x,y}
        loss_weights[output_weights[index]]=weights_vals_dist
        #if equalize_classes == True:
        #    expanded_df = equalize_classwise_dist(train_df, selector_column, count)
        #    train_df = train_df.append(expanded_df, ignore_index=True)
        index+=1
    #print(loss_weights)
    return train_df,loss_weights

In [0]:
import numpy as np
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()
        if p_1 > p:
            return input_img
        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)
            if left + w <= img_w and top + h <= img_h:
                break
        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c
        return input_img
    return eraser

In [0]:
#test_df = get_dist(test_df, equalize_classes=True)
test_df = train_df
_,loss_weights_train=get_dist(test_df, equalize_classes=False)
loss_weights_train

['gender_female', 'gender_male']
[1.286, 1.0]
1.286 0
1.0 1
['imagequality_Average', 'imagequality_Bad', 'imagequality_Good']
[1.0, 3.32, 1.973]
1.0 0
3.32 1
1.973 2
['age_15-25', 'age_25-35', 'age_35-45', 'age_45-55', 'age_55+']
[2.172, 1.0, 1.579, 3.626, 7.27]
2.172 0
1.0 1
1.579 2
3.626 3
7.27 4
['weight_normal-healthy', 'weight_over-weight', 'weight_slightly-overweight', 'weight_underweight']
[1.0, 9.917, 2.671, 10.109]
1.0 0
9.917 1
2.671 2
10.109 3
['carryingbag_Daily/Office/Work Bag', 'carryingbag_Grocery/Home/Plastic Bag', 'carryingbag_None']
[1.671, 5.756, 1.0]
1.671 0
5.756 1
1.0 2
['footwear_CantSee', 'footwear_Fancy', 'footwear_Normal']
[1.192, 2.427, 1.0]
1.192 0
2.427 1
1.0 2
['emotion_Angry/Serious', 'emotion_Happy', 'emotion_Neutral', 'emotion_Sad']
[6.401, 6.217, 1.0, 12.141]
6.401 0
6.217 1
1.0 2
12.141 3
['bodypose_Back', 'bodypose_Front-Frontish', 'bodypose_Side']
[3.751, 1.0, 2.78]
3.751 0
1.0 1
2.78 2


{'age_output': {0: 2.172, 1: 1.0, 2: 1.579, 3: 3.626, 4: 7.27},
 'bag_output': {0: 1.671, 1: 5.756, 2: 1.0},
 'emotion_output': {0: 6.401, 1: 6.217, 2: 1.0, 3: 12.141},
 'footwear_output': {0: 1.192, 1: 2.427, 2: 1.0},
 'gender_output': {0: 1.286, 1: 1.0},
 'imagequality_ouput': {0: 1.0, 1: 3.32, 2: 1.973},
 'pose_output': {0: 3.751, 1: 1.0, 2: 2.78},
 'weight_output': {0: 1.0, 1: 9.917, 2: 2.671, 3: 10.109}}

In [0]:
BATCH_SIZE=16

aug_gen = ImageDataGenerator(horizontal_flip=True, 
                             vertical_flip=False,
                             rotation_range=5,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=[0.5,1.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,1.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             )
                             #batch_size=BATCH_SIZE)

train_gen = PersonDataGenerator(train_df, batch_size=BATCH_SIZE,normalize=True,aug_flow=aug_gen)
valid_gen = PersonDataGenerator(val_df, batch_size=BATCH_SIZE, shuffle=False,normalize=True)

In [0]:
train_gen[0]

In [0]:
def get_image_batch(data_df, batch_size=32, shuffle=True,normalize=True, selected_field='age_output'):
    new_batch = PersonDataGenerator(data_df, batch_size,shuffle, normalize)
    images, targets = next(iter(new_batch))
    num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
    labels = np.asarray([ np.argmax(targets['age_output'][pos]) for pos in range(len(targets['age_output'])) ])
    return images,labels, targets, len(images)

In [0]:
images, y_train, targets, len_train = get_image_batch(train_df, batch_size=32,normalize=True, shuffle=True)

In [0]:
images_test, y_test, targets_test, len_test = get_image_batch(val_df, batch_size=32,normalize=True, shuffle=False)

In [0]:
def print_mean_std_for_batch(datagen_process):
    image_val,target = next(iter(datagen_process))
    print(image_val.shape)
    print(np.mean(image_val.round(2), axis=(0,1,2)),np.std(image_val.round(2), axis=(0,1,2)) )
    #print(np.mean(image_val.round(2), axis=(0,1,2)),np.std(image_val.round(2), axis=(0,1,2)) )  


In [0]:
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

images_test, targets_test = next(iter(valid_gen))

print(num_units)
#print(np.mean(images.round(2), axis=(0,1,2)),np.std(images.round(2), axis=(0,1,2)) )
#print(np.mean(images_test.round(2), axis=(0,1,2)),np.std(images_test.round(2), axis=(0,1,2)) )

print_mean_std_for_batch(train_gen)
print_mean_std_for_batch(valid_gen)

{'gender': 2, 'image_quality': 3, 'age': 5, 'weight': 4, 'bag': 3, 'pose': 3, 'footwear': 3, 'emotion': 4}
(16, 224, 224, 3)
[-0.0005297   0.00061987  0.00060541] [1.0002795 0.9988922 1.0019801]
(16, 224, 224, 3)
[-3.1902869e-06  3.5649276e-04 -2.6717642e-04] [1.0012239 1.0019708 1.0025381]


In [0]:
num_units={'age': 5}

In [0]:
def display_single_image(image):
    cv2_imshow(cv2.resize(image, (image.shape[1], image.shape[0])))

In [0]:
#cv2_imshow(cv2.resize(images_norm[0], (images_norm[0].shape[1], images_norm[0].shape[0])))
#images_norm.shape
#images_test[0].shape
display_single_image(images[10])
#display_single_image(images_test_norm[10])

In [0]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import time,math
############# Weights initializer #################
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K

In [0]:
tf.compat.v1.disable_eager_execution()

In [0]:
BATCH_SIZE=32
LEARNING_RATE=2
EPOCHS=50
MOMENTUM=0.9
train_df.shape
batches_per_epoch = train_df.shape[0]//BATCH_SIZE + 1
batches_per_epoch

In [0]:
# import the necessary packages
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers import GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.core import Activation
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.layers import add
from keras.regularizers import l2
from keras import backend as K

In [0]:
epsilon_c=1e-5
momentum_c=0.9
class ResNet_DavidNet:
    ## Static Values


    @staticmethod
    def conv_builder(input_data, 
                     channels_out, 
                     kernel_size=(3,3), 
                     padding="SAME", 
                     strides=(1,1), 
                     use_dropout=True, 
                     activation=None,
                     kernel_initializer='he_normal',
                     separable=False):
        if (separable == True):
            x = SeparableConv2D(filters=channels_out,
                kernel_size=kernel_size, 
                padding=padding,
                strides=strides, 
                use_bias=False)(input_data)
        else:    
            x = Conv2D(filters=channels_out,
                    kernel_size=kernel_size, 
                    padding=padding,
                    strides=strides, 
                    use_bias=False )(input_data)

        if use_dropout:
            x = Dropout(0.075)(x)

        x = BatchNormalization(epsilon=epsilon_c, momentum=momentum_c)(x)

        if activation != None:
            x =  Activation(activation)(x)
        
        return x
    @staticmethod
    ### Builds a stack of
    #### (3x3)|(3x3 Stride=2)|MaxPool2D 
    def build_conv_max_pool_blocks(inputs_data, 
                                    channels_out, 
                                    use_dropout=True, 
                                    activation=None,                                    
                                    separable=False,
                                    pool=True):
        
        conv_layer = ResNet_DavidNet.conv_builder(inputs_data,
                                        channels_out,
                                        use_dropout=True,
                                        activation=activation,
                                        kernel_initializer='he_normal',
                                        separable=separable)                                      
        conv_layer = ResNet_DavidNet.conv_builder(conv_layer,
                                        channels_out,
                                        use_dropout=True,
                                        activation=activation,
                                        kernel_initializer='he_normal',
                                        strides=(2,2),
                                        separable=separable)
        if pool == True:
            conv_layer = MaxPooling2D()(conv_layer)
        
        return conv_layer
        

    @staticmethod
    def residual_block(inputs_data, 
                       channels_out, 
                       pool_layer=False, 
                       res=False,
                       separable=False
                      ):
        #shortcut=data
        convBn_1 = ResNet_DavidNet.conv_builder(inputs_data,                                
                                channels_out,
                                kernel_size=(1,1),
                                strides=(2,2),
                                use_dropout=True,
                                activation='relu',
                                separable=separable
                                )

        convBn = ResNet_DavidNet.conv_builder(convBn_1,
                                        channels_out,
                                        use_dropout=True,
                                        activation='relu',
                                        separable=separable
                                        )
        if res:
            # Add two Conv Layers
            x = ResNet_DavidNet.conv_builder(convBn, 
                                        channels_out,
                                        use_dropout=True,
                                        activation='relu',
                                        separable=separable
                                        )
            x = ResNet_DavidNet.conv_builder(x, 
                                        channels_out,
                                        use_dropout=True,
                                        activation='relu',
                                        separable=separable
                                        )
            convBn = add([convBn_1, x])
        if pool_layer:
            convBn = MaxPooling2D()(convBn)
            convBn = MaxPooling2D()(convBn)   
        ### Return the constructed layer
        return convBn
    @staticmethod
    def get_onexone_conv(attribute_str, input_layer,num_units_in):
        attr_name = attribute_str+'_output'

        return Flatten(name=attr_name)((ResNet_DavidNet.conv_builder(input_layer,
                                                                     channels_out=5,
                                                                     kernel_size=(1,1),
                                                                     strides=(1,1),
                                                                     activation='softmax',
                                                                     kernel_initializer='he_normal')(input_layer)))
    @staticmethod
    def build(inputShape, 
              num_units_in=num_units, 
              channels_out=64, 
              output_field=None, 
              num_layers=3,
              separable = False):
    #		reg=0.0001, bnEps=2e-5, bnMom=0.9, dataset="cifar"):
        
        inputs_data = Input(shape=inputShape)
        print(inputs_data)

        model = ResNet_DavidNet.conv_builder(inputs_data, 
                                             channels_out,
                                             use_dropout=True,
                                             activation='relu',
                                             separable=separable)
        #inputs_data = model
        # model = ResNet_DavidNet.build_conv_max_pool_blocks(model, 
        #                                                    channels_out, 
        #                                                    use_dropout=True, 
        #                                                    activation='relu',                                    
        #                                                   separable=separable,
        #                                                   pool=True )
        # #inputs_data = model
        # model = ResNet_DavidNet.build_conv_max_pool_blocks(model, 
        #                                                    channels_out*2, 
        #                                                    use_dropout=True, 
        #                                                    activation='relu',                                    
        #                                                   separable=separable,
        #                                                   pool=True )
        # model = ResNet_DavidNet.build_conv_max_pool_blocks(model, 
        #                                                    channels_out*4, 
        #                                                    use_dropout=True, 
        #                                                    activation='relu',                                    
        #                                                   separable=separable,
        #                                                   pool=True )
        # model = ResNet_DavidNet.build_conv_max_pool_blocks(model, 
        #                                                    channels_out*8, 
        #                                                    use_dropout=True, 
        #                                                    activation='relu',                                    
        #                                                   separable=separable,
        #                                                   pool=True )

        # model = ResNet_DavidNet.conv_builder(inputs_data, 
        #                                      channels_out,
        #                                      use_dropout=True,
        #                                      activation='relu',
        #                                      separable=separable) #
        model = ResNet_DavidNet.residual_block(model, 
                                               channels_out*2, 
                                               pool_layer=True, 
                                               res=True,
                                               separable=separable)
        
        model = ResNet_DavidNet.residual_block(model, 
                                                channels_out*4,
                                                pool_layer=True,
                                                res=True,
                                               separable=separable)
        # model = ResNet_DavidNet.residual_block(model, 
        #                                         channels_out*4,
        #                                         pool_layer=True,
        #                                         res=True,
        #                                        separable=separable)
        # model = ResNet_DavidNet.residual_block(model, 
        #                                         channels_out*4,
        #                                         pool_layer=True,
        #                                         res=True,
        #                                        separable=separable)
        # model = ResNet_DavidNet.residual_block(model, 
        #                                         channels_out*8,
        #                                         pool_layer=True,
        #                                         res=True,
        #                                        separable=separable)
        model = AveragePooling2D()(model)
        model = GlobalMaxPooling2D()(model)
        # #model = GlobalAveragePooling2D()(model)
        # #model = Dense(128, activation="relu")(model)
        # #model = Dropout(0.3)(model)
        if(output_field == None):
            output_vals = [Dense(num_units[name], activation="softmax", name=f"{name}_output")(model) for name in num_units.keys()]
        else:
            output_vals = Dense(num_units[output_field], activation="softmax", name=f"{output_field}_output")(model)

        #output_vals = Flatten(name="flatten")(output_vals)
        #Flatten(name=attr_name)
        
        #ResNet_DavidNet.get_onexone_conv('age',model,num_units_in) #for index in num_units_in.keys()]
        model = Model(inputs=inputs_data, outputs=output_vals)

        return model

In [0]:
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)#,output_field='age') 
                    #    num_units_in=num_units, 
                    #    64)
modelv2.summary()

In [0]:
from keras.utils import np_utils
from keras.utils import plot_model
plot_model(modelv2, 'latest.png', show_shapes=True)

In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = False

In [0]:
#wrn_28_10=create_model()
from keras.models import load_model
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

# class LossHistory(keras.callbacks.Callback):
# 	def on_train_begin(self, logs={}):
# 		print("Clearing saved content on training start")
# 		self.losses = []
# 		self.best = np.Inf

class TrainingMonitor(BaseLogger):
	def __init__(self, figPath, jsonPath=None, startAt=0, backup_hist=True):
		# store the output path for the figure, the path to the JSON
		# serialized file, and the starting epoch
		super(TrainingMonitor, self).__init__()
		self.figPath = figPath
		self.jsonPath = jsonPath
		self.startAt = startAt
		self.backup_hist = backup_hist
		print("JSON path:",self.jsonPath)

	def on_train_begin(self, logs={}):
		# initialize the history dictionary
		self.H = {}
		#self.losses = []

		# if the JSON history path exists, load the training history
		if self.jsonPath is not None:
			if os.path.exists(self.jsonPath) and (self.backup_hist == True):
				#self.H = json.loads(open(self.jsonPath).read())
				backup_file_name=self.jsonPath+str(get_curr_time())+"_backup"
				print("Backing up history file:",self.jsonPath," to:",backup_file_name)
				os.rename(self.jsonPath,backup_file_name) 

				# # check to see if a starting epoch was supplied
				# if self.startAt > 0:
				# 	# loop over the entries in the history log and
				# 	# trim any entries that are past the starting
				# 	# epoch
				# 	for k in self.H.keys():
				# 		self.H[k] = self.H[k][:self.startAt]

	def on_epoch_end(self, epoch, logs={}):
		# loop over the logs and update the loss, accuracy, etc.
		# for the entire training process
		for (k, v) in logs.items():
			l = self.H.get(k, [])
			l.append(float(v))
			self.H[k] = l

		# check to see if the training history should be serialized
		# to file
		if self.jsonPath is not None:
			f = open(self.jsonPath, "w")
			f.write(json.dumps(self.H))
			f.close()
	def on_train_end(self, logs={}):
		print("Saved JSON PATH:",self.jsonPath)

import os
plotPath = png_file
jsonPath = json_file
print(plotPath,jsonPath)



/content/gdrive/My Drive/DavNet_Base/png_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.png /content/gdrive/My Drive/DavNet_Base/json_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.json


In [0]:
history=[0]*100

In [0]:
#0.30007923
#0.25668284
## Stable --->
#Min: 7.108769 0.22178908
#Min: 7.2756057 0.12301114
#Min: 7.295935 0.24042512
#Min: 7.7571077 0.030115802
#Min: 7.5900717 0.01655656
#Min: 7.1677036 0.21655203
#Min: 7.195501 0.10309692
#Min: 8.308885 0.09440613
del modelv2
MOMENTUM=0.9
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)
#modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=32)
modelv2.compile(
    #optimizer=SGD(lr=2.2915473*0.01,momentum=MOMENTUM, nesterov=True),
    optimizer=SGD(lr=0.23201092*0.1),
    loss=tf.keras.losses.CategoricalCrossentropy(),     
    #weighted_metrics=["accuracy"]
    metrics=["accuracy"]
)
lr_finder = LRFinder_new(modelv2)
lr_finder.find_generator(train_gen,start_lr=0.001, end_lr=0.1,epochs=20
                         ,steps_per_epoch=20)
                         #,class_weight=class_weight)

#print(lr_finder.lrs[np.argmax(lr_finder.losses)])
#print(lr_finder.lrs[np.argmin(lr_finder.losses)])

print("#Max:", np.max(lr_finder.losses),lr_finder.lrs[np.argmax(lr_finder.losses)])
print("#Min:", np.min(lr_finder.losses), lr_finder.lrs[np.argmin(lr_finder.losses)])

In [0]:
print("Max:", np.max(lr_finder.losses),lr_finder.lrs[np.argmax(lr_finder.losses)])
print("Min:", np.min(lr_finder.losses), lr_finder.lrs[np.argmin(lr_finder.losses)])

In [0]:
lr_finder.plot_loss(n_skip_beginning=2, n_skip_end=1)

In [0]:
print(lr_finder.get_best_lr(1,1,1))
print(lr_finder.get_best_lr(10,1,1))
print(lr_finder.get_best_lr(20,1,1))

lrs_to_losses = pd.DataFrame({'lrs':lr_finder.lrs, 'losses':lr_finder.losses})
lrs_to_losses.loc[lrs_to_losses.shape[0]-1]
sma_10_lr = lr_finder.get_best_lr(10,1,1)
lrs_to_losses.loc[lrs_to_losses[lrs_to_losses['lrs'].between(sma_10_lr*0.1, sma_10_lr) ]['losses'].idxmin()]

In [0]:
lrs_to_losses

In [0]:
lrs_to_losses_10 = pd.DataFrame({'lrs':lr_finder.lrs, 'losses':lr_finder.losses})
lrs_to_losses_10.describe()

In [0]:
lrs_to_losses.describe()

In [0]:
lrs_to_losses_100 = pd.DataFrame({'lrs':lr_finder.lrs, 'losses':lr_finder.losses})
lrs_to_losses_100.describe()

In [0]:
len(lr_finder.losses)

In [0]:
print(np.min(lr_finder.losses))
#(lr_finder.losses)

In [0]:
print(lr_finder.lrs[np.argmax(lr_finder.losses)])
print(lr_finder.lrs[np.argmin(lr_finder.losses)])

In [0]:
np.max(lr_finder.losses)

In [0]:
lr_finder.plot_loss(n_skip_end=1,x_scale='linear')

In [0]:
lr_finder.plot_loss_change(sma=1, n_skip_beginning=1, n_skip_end=1, y_lim=(np.min(lr_finder.get_derivatives(sma=1)), np.max(lr_finder.get_derivatives(sma=1))))

In [0]:
derivatives_2 = lr_finder.get_derivatives(sma=10)

In [0]:
EPOCHS

In [0]:
test_y=np.linspace(0, np.int(EPOCHS), np.int(EPOCHS))

In [0]:
# from datetime import datetime
# # Prepare model model saving directory.
# import os
# save_dir = os.path.join('/content/gdrive/', 'My Drive')
# def get_curr_time():
#     return int(datetime.utcnow().strftime("%s"))
# model_name = 'assignment5_%s_model.{epoch:03d}.h5' % (model_name_itr)
# print(model_name)
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# filepath = os.path.join(save_dir, model_name)

# # Prepare callbacks for model saving and for learning rate adjustment.
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_loss',
#                              verbose=1,
#                              save_best_only=True)

In [0]:
from datetime import datetime
# Prepare model model saving directory.
import os
save_dir = os.path.join('/content/gdrive/', 'My Drive/DavNet_Base')
def get_curr_time():
    return int(datetime.utcnow().strftime("%s"))
# model_name = 'a5_%s_model.{epoch:03d}.h5' % (model_name_itr)
# print(model_name)
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)

# filepath = os.path.join(save_dir, model_name)
# print(filepath)

def generate_new_callbacks(steps_per_epoch=50,epoch_count=50,min_lr=0.00001, max_lr=0.1,patience=25):
    model_name = 'a5_%s_model.{epoch:03d}.h5' % (model_name_itr+"_"+str(get_curr_time()))
    print(model_name)
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)

    filepath = os.path.join(save_dir, model_name)
    print(filepath)
    local_clr = CyclicLR(base_lr=min_lr,
                            max_lr=max_lr,
                            step_size=steps_per_epoch*4,
                            mode='triangular')
    checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')
    training_mon = TrainingMonitor(figPath=plotPath,
                                   jsonPath=jsonPath,
                                   startAt=0)
    ########## Introduced after 2x100 Epochs
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                    patience=patience, 
                                                    restore_best_weights=True)
    print("Returning new callback array with steps_per_epoch=",steps_per_epoch,
          "min_lr=",min_lr,
          "max_lr=",max_lr,
          "epoch_count=",epoch_count
          )
    callback_array = [local_clr, checkpoint, training_mon, early_stop]
    return callback_array

In [0]:
LEARNING_RATE=0.052480783*0.1
#*0.1
EPOCHS=50
test_y = np.linspace(0,EPOCHS,EPOCHS)
x=[0, (EPOCHS+1)//5, EPOCHS]
#y=[LEARNING_RATE*0.01, LEARNING_RATE, LEARNING_RATE*0.01]
y=[LEARNING_RATE*0.01, LEARNING_RATE, LEARNING_RATE*0.01]
interp_lr = np.interp(test_y, x, y)
def one_lr_schedule(epoch):
    print("lr:",interp_lr[epoch])
    return interp_lr[epoch]
#interp_values = np.interp(, [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])
lr_scheduler = LearningRateScheduler(one_lr_schedule)
callbacks = [checkpoint, lr_scheduler,TrainingMonitor(figPath=plotPath,
                                                      jsonPath=jsonPath,startAt=2)]

In [0]:
plt.plot(test_y,interp_lr)

In [0]:
import gc
def reset_keras(per_process_gpu_memory_fraction=1.0):
    """
    Reset Keras session and set GPU configuration as well as collect unused memory.
    This is adapted from [jaycangel's post on fastai forum](https://forums.fast.ai/t/how-could-i-release-gpu-memory-of-keras/2023/18).
    Calling this before any training will clear Keras session. Hence, a Keras model must be redefined and compiled again.
    It can be used in during hyperparameter scan or K-fold validation when model training is invoked several times.
    :param per_process_gpu_memory_fraction: tensorflow's config.gpu_options.per_process_gpu_memory_fraction
    """
    sess = K.get_session()
    K.clear_session()
    sess.close()

    gc.collect()

    # use the same config as you used to create the session
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = per_process_gpu_memory_fraction
    config.gpu_options.visible_device_list = "0"
    K.set_session(tf.Session(config=config))

In [0]:
reset_keras()

In [0]:
history=[0]*200

In [0]:
# #Min: 7.108769 0.22178908
# #Min: 7.2756057 0.12301114
# #Min: 7.295935 0.24042512
# #Min: 7.7571077 0.030115802
# #Min: 7.5900717 0.01655656
# #del modelv2
# MOMENTUM=0.9
# #LEARNING_RATE=0.21655203
# #LEARNING_RATE=6.7237797*0.1-----> no learning
# #LEARNING_RATE=0.5037587*0.1
# #optimizer=SGD(lr=LEARNING_RATE)
# input_shape=(224,224,3)

# optimizer=SGD(lr=one_lr_schedule(0))
# #0.101295
# #*0.1
# #0.22178908
# #loss_weights_compile = {'gender_output': 2, 'image_quality_output': 3, 'age_output': 5, 'weight_output': 4, 'bag_output': 3, 'pose_output': 3, 'footwear_output': 3, 'emotion_output': 4}
# modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)
# modelv2.load_weights('/content/gdrive/My Drive/assignment5_davnetsep_widrn_gml2_1577343325_model.025.h5')
# modelv2.compile(
#     #optimizer=SGD(lr=0.22178908*0.01,momentum=MOMENTUM, nesterov=True),
#     optimizer=optimizer,
#     loss=tf.keras.losses.CategoricalCrossentropy(),
#     #loss_weights=loss_weights_compile,
#     #weighted_metrics=["accuracy"]
#     metrics=["accuracy"]
# )

In [0]:
#LEARNING_RATE=0.020183668*0.15
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=30
EPOCHS=100
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

Tensor("input_5:0", shape=(?, 224, 224, 3), dtype=float32)
JSON path: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json
Returning new callback array with steps_per_epoch= 30 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 100


Epoch 1/100
30/30 [==============================] - 34s 1s/step - loss: 11.4476 - gender_output_loss: 1.2359 - image_quality_output_loss: 1.3282 - age_output_loss: 2.4976 - weight_output_loss: 1.1581 - bag_output_loss: 1.0795 - pose_output_loss: 1.4303 - footwear_output_loss: 1.1128 - emotion_output_loss: 1.6052 - gender_output_acc: 0.5583 - image_quality_output_acc: 0.4125 - age_output_acc: 0.3125 - weight_output_acc: 0.5396 - bag_output_acc: 0.5042 - pose_output_acc: 0.3958 - footwear_output_acc: 0.4375 - emotion_output_acc: 0.4875 - val_loss: 10.2491 - val_gender_output_loss: 0.7290 - val_image_quality_output_loss: 1.5273 - val_age_output_loss: 1.7472 - val_weight_output_loss: 1.4363 - val_bag_output_loss: 1.0200 - val_pose_o

In [0]:
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=100
EPOCHS=100
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

Tensor("input_6:0", shape=(?, 224, 224, 3), dtype=float32)
JSON path: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json
Returning new callback array with steps_per_epoch= 100 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 100
Backing up history file: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json  to: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json1577482418_backup
Epoch 1/100
100/100 [==============================] - 49s 491ms/step - loss: 11.4395 - gender_output_loss: 0.8320 - image_quality_output_loss: 1.1172 - age_output_loss: 3.3260 - weight_output_loss: 1.3231 - bag_output_loss: 0.9946 - pose_output_loss: 1.3749 - footwear_output_loss: 1.1836 - emotion_output_loss: 1.2881 - gender_output_acc: 0.5212 - image_quality_output_acc: 0.4944 - age_output_acc: 0.2325 - weight_output_acc: 0.5787 - bag_output_acc: 0.5438 - pose_output_acc: 0.5787 - footwear_output_acc: 0.4025 - emotion_output_acc: 0.63

In [0]:
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=100
EPOCHS=100
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

Tensor("input_7:0", shape=(?, 224, 224, 3), dtype=float32)
JSON path: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json
Returning new callback array with steps_per_epoch= 100 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 100
Backing up history file: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json  to: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577479189.json1577487426_backup
Epoch 1/100
100/100 [==============================] - 50s 499ms/step - loss: 10.5970 - gender_output_loss: 0.8408 - image_quality_output_loss: 1.1708 - age_output_loss: 2.1070 - weight_output_loss: 1.1201 - bag_output_loss: 1.2838 - pose_output_loss: 1.3664 - footwear_output_loss: 1.2179 - emotion_output_loss: 1.4902 - gender_output_acc: 0.5206 - image_quality_output_acc: 0.4706 - age_output_acc: 0.2969 - weight_output_acc: 0.6244 - bag_output_acc: 0.5281 - pose_output_acc: 0.4944 - footwear_output_acc: 0.4031 - emotion_output_acc: 0.60

In [0]:
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=200
EPOCHS=50
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)

#### Last saved model
modelv2.load_weights('/content/gdrive/My Drive/DavNet_Base/assignment5_davnetsep_widrn_gml2_wrn_ddd_1577479189_model.046.h5')
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

Tensor("input_4:0", shape=(?, 224, 224, 3), dtype=float32)
JSON path: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577512907.json
Returning new callback array with steps_per_epoch= 200 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 50


Epoch 1/50
200/200 [==============================] - 93s 463ms/step - loss: 7.8438 - gender_output_loss: 0.6735 - image_quality_output_loss: 0.9597 - age_output_loss: 1.4405 - weight_output_loss: 1.0059 - bag_output_loss: 0.9191 - pose_output_loss: 0.9427 - footwear_output_loss: 1.0108 - emotion_output_loss: 0.8916 - gender_output_acc: 0.5834 - image_quality_output_acc: 0.5469 - age_output_acc: 0.3875 - weight_output_acc: 0.6206 - bag_output_acc: 0.5413 - pose_output_acc: 0.6059 - footwear_output_acc: 0.4953 - emotion_output_acc: 0.7194 - val_loss: 7.8694 - val_gender_output_loss: 0.6698 - val_image_quality_output_loss: 0.9290 - val_age_output_loss: 1.4187 - val_weight_output_loss: 1.0057 - val_bag_output_loss: 0.9238 - val_pose

In [0]:
train_df.shape[0]//BATCH_SIZE//2

360

In [0]:
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE//2
EPOCHS=50
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
#modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)

#### Last saved model
modelv2.load_weights('/content/gdrive/My Drive/assignment5_davnetsep_widrn_gml2_wrn_ddd_1577512907_model.048.h5')
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

JSON path: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577512907.json
Returning new callback array with steps_per_epoch= 360 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 50
Epoch 1/50
360/360 [==============================] - 142s 395ms/step - loss: 7.6767 - gender_output_loss: 0.6400 - image_quality_output_loss: 0.9486 - age_output_loss: 1.4001 - weight_output_loss: 0.9899 - bag_output_loss: 0.8978 - pose_output_loss: 0.9079 - footwear_output_loss: 0.9929 - emotion_output_loss: 0.8997 - gender_output_acc: 0.6281 - image_quality_output_acc: 0.5396 - age_output_acc: 0.4062 - weight_output_acc: 0.6286 - bag_output_acc: 0.5646 - pose_output_acc: 0.6222 - footwear_output_acc: 0.5193 - emotion_output_acc: 0.7120 - val_loss: 7.7081 - val_gender_output_loss: 0.6218 - val_image_quality_output_loss: 0.8999 - val_age_output_loss: 1.4021 - val_weight_output_loss: 0.9919 - val_bag_output_loss: 0.9068 - val_pose_output_loss: 0.9007 - val_footwear_output_loss: 1.0404 - va

In [0]:
print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

End of EPOCHS= 50  STEPS_PER_EPOCH= 360


In [0]:
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE
EPOCHS=30
# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
#modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)

#### Last saved model
modelv2.load_weights('/content/gdrive/My Drive/assignment5_davnetsep_widrn_gml2_wrn_ddd_1577512907_model.042.h5')
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

JSON path: /content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_ddd_1577512907.json
Returning new callback array with steps_per_epoch= 721 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 30
Epoch 1/30
721/721 [==============================] - 266s 369ms/step - loss: 7.4951 - gender_output_loss: 0.6059 - image_quality_output_loss: 0.9237 - age_output_loss: 1.3965 - weight_output_loss: 0.9669 - bag_output_loss: 0.8872 - pose_output_loss: 0.8719 - footwear_output_loss: 0.9523 - emotion_output_loss: 0.8906 - gender_output_acc: 0.6660 - image_quality_output_acc: 0.5609 - age_output_acc: 0.4002 - weight_output_acc: 0.6340 - bag_output_acc: 0.5834 - pose_output_acc: 0.6211 - footwear_output_acc: 0.5456 - emotion_output_acc: 0.7143 - val_loss: 7.6472 - val_gender_output_loss: 0.5768 - val_image_quality_output_loss: 0.9373 - val_age_output_loss: 1.4075 - val_weight_output_loss: 1.0037 - val_bag_output_loss: 0.8819 - val_pose_output_loss: 0.8152 - val_footwear_output_loss: 1.0797 - va

In [0]:
LEARNING_RATE=0.052480783*0.1
STEPS_PER_EPOCH=train_df.shape[0]//BATCH_SIZE
EPOCHS=100
ATTEMPT_COUNTER=1
aug_gen = ImageDataGenerator(horizontal_flip=False, 
                             vertical_flip=False,
                             rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.1,
                             zoom_range=[0.5,2.5],
                             shear_range=0.2,
                             #zca_whitening=True,
                             brightness_range=[0.5,3.5],
                             #preprocessing_function=get_random_eraser(v_l=0, v_h=1, pixel_level=False)
                             )

train_gen = PersonDataGenerator(train_df, batch_size=BATCH_SIZE,normalize=True,aug_flow=aug_gen)
valid_gen = PersonDataGenerator(val_df, batch_size=BATCH_SIZE, shuffle=False,normalize=True)
model_name_itr = model_name_itr+str(ATTEMPT_COUNTER)
print("Model_Name")

# loss_weights_compile = {'gender_output': 2, 
#                         'image_quality_output': 2, 
#                         'age_output': 4, 
#                         'weight_output': 3, 
#                         'bag_output': 3, 
#                         'pose_output': 3, 
#                         'footwear_output': 2, 
#                         'emotion_output': 4}
input_shape=(224,224,3)
modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=64,separable=True)

#### Last saved model
modelv2.load_weights('/content/gdrive/My Drive/DavNet_Base/a5_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_15775476121_1577548095_model.001.h5')
#print(modelv2.best)
#modelv2.
modelv2.compile(
     #optimizer=SGD(lr=1.3513402*0.1),
     optimizer=SGD(lr=0.0001),
     loss=tf.keras.losses.CategoricalCrossentropy(),
     #loss_weights=loss_weights_compile,
     #weighted_metrics=["accuracy"]
     metrics=["accuracy"]
)
callbacks = generate_new_callbacks(steps_per_epoch=STEPS_PER_EPOCH, 
                                     epoch_count=EPOCHS,
                                     min_lr=LEARNING_RATE*0.01, 
                                     max_lr=LEARNING_RATE, patience=10)
#print(callbacks)
modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=4, 
    epochs=EPOCHS,
    verbose=1,
    #class_weight=loss_weights_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)

print("End of EPOCHS=",EPOCHS," STEPS_PER_EPOCH=",STEPS_PER_EPOCH)

Model_Name
Tensor("input_4:0", shape=(?, 224, 224, 3), dtype=float32)
a5_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_15775476121111_1577553601_model.{epoch:03d}.h5
/content/gdrive/My Drive/DavNet_Base/a5_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_15775476121111_1577553601_model.{epoch:03d}.h5
JSON path: /content/gdrive/My Drive/DavNet_Base/json_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.json
Returning new callback array with steps_per_epoch= 721 min_lr= 5.2480783e-05 max_lr= 0.0052480783 epoch_count= 100
Backing up history file: /content/gdrive/My Drive/DavNet_Base/json_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.json  to: /content/gdrive/My Drive/DavNet_Base/json_davnetsep_widrn_gml2_wrn_ddd_sth_rjy_1577547612.json1577553604_backup
Epoch 1/100
721/721 [==============================] - 478s 663ms/step - loss: 7.7043 - gender_output_loss: 0.6445 - image_quality_output_loss: 0.9611 - age_output_loss: 1.4196 - weight_output_loss: 0.9766 - bag_output_loss: 0.9071 - pose_output_loss: 0.9089 - 

In [0]:
!cp "/content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_1577374571.json" "'/content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_1577374571_round1.json'"

In [0]:
history[0]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=EPOCHS,
    callbacks=callbacks,
    #class_weight=loss_weights_train,
    verbose=1
)

In [0]:
!cp "/content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_1577374571.json" "/content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_1577374571_round2.json"

In [0]:
history[0]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=EPOCHS,
    callbacks=callbacks,
    #class_weight=loss_weights_train,
    verbose=1
)

In [0]:
!cp "/content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_1577374571.json" "/content/gdrive/My Drive/json_davnetsep_widrn_gml2_wrn_1577374571_round3.json"

In [0]:
history[0]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=EPOCHS,
    callbacks=callbacks,
    #class_weight=loss_weights_train,
    verbose=1
)

In [0]:
!cp "/content/gdrive/My Drive/assignment5_simplecnn2_widrn_acc_1577201016_model.047.h5" "/content/gdrive/My Drive/simplecnn2_widrn_round1"

In [0]:
!cp "/content/gdrive/My Drive/json_simplecnn2_widrn_acc_1577201016.json" "/content/gdrive/My Drive/back_json"

In [0]:
history[1]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=50,
    callbacks=callbacks,
    class_weight=loss_weights_train,
    verbose=1
)

In [0]:
modelv2.save_weights('/content/gdrive/My Drive/h5_simplecnn2_widrn_acc_1577201016.h5')

In [0]:
!cp "/content/gdrive/My Drive/json_simplecnn2_widrn_acc_1577201016.json" "/content/gdrive/My Drive/back_json"

In [0]:
history = [0] * 100


In [0]:
# modelv2 = ResNet_DavidNet.build(inputShape=input_shape,channels_out=32)
# modelv2.compile(
#     #optimizer=SGD(lr=2.2915473*0.01,momentum=MOMENTUM, nesterov=True),
#     optimizer=SGD(lr=0.23201092*0.1),
#     loss=tf.keras.losses.CategoricalCrossentropy(),     
#     weighted_metrics=["accuracy"]
# )
history[2]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=50,
    callbacks=callbacks,
    class_weight=loss_weights_train,
    verbose=1
)

In [0]:
history[3]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=50,
    callbacks=callbacks,
    class_weight=loss_weights_train,
    verbose=1
)

In [0]:
history[3]=modelv2.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=False,
    workers=4, 
    epochs=50,
    callbacks=callbacks,
    class_weight=loss_weights_train,
    verbose=1
)

In [0]:
from matplotlib import pyplot as plt
import math
from keras.callbacks import LambdaCallback
import keras.backend as K
import numpy as np


class LRFinder_new:
    """
    Plots the change of the loss function of a Keras model when the learning rate is exponentially increasing.
    See for details:
    https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0
    """

    def __init__(self, model):
        self.model = model
        self.losses = []
        self.lrs = []
        self.best_loss = 1e9

    def on_batch_end(self, batch, logs):
        # Log the learning rate
        lr = K.get_value(self.model.optimizer.lr)
        self.lrs.append(lr)

        # Log the loss
        loss = logs['loss']
        self.losses.append(loss)

        # Check whether the loss got too large or NaN
        if batch > 5 and (math.isnan(loss) or loss > self.best_loss * 4):
            self.model.stop_training = True
            print("")
            print("Training halted as loss reached maximum",loss)
            return

        if loss < self.best_loss:
            self.best_loss = loss

        # Increase the learning rate for the next batch
        lr *= self.lr_mult
        K.set_value(self.model.optimizer.lr, lr)

    def find(self, x_train, y_train, start_lr, end_lr, batch_size=64, epochs=1):
        # If x_train contains data for multiple inputs, use length of the first input.
        # Assumption: the first element in the list is single input; NOT a list of inputs.
        N = x_train[0].shape[0] if isinstance(x_train, list) else x_train.shape[0]

        # Compute number of batches and LR multiplier
        num_batches = epochs * N / batch_size
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(num_batches))
        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))

        self.model.fit(x_train, y_train,
                       batch_size=batch_size, epochs=epochs,
                       callbacks=[callback])

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def find_generator(self, generator, start_lr, end_lr, epochs=1, steps_per_epoch=None, **kw_fit):
        if steps_per_epoch is None:
            try:
                steps_per_epoch = len(generator)
            except (ValueError, NotImplementedError) as e:
                raise e('`steps_per_epoch=None` is only valid for a'
                        ' generator based on the '
                        '`keras.utils.Sequence`'
                        ' class. Please specify `steps_per_epoch` '
                        'or use the `keras.utils.Sequence` class.')
        self.lr_mult = (float(end_lr) / float(start_lr)) ** (float(1) / float(epochs * steps_per_epoch))

        # Save weights into a file
        self.model.save_weights('tmp.h5')

        # Remember the original learning rate
        original_lr = K.get_value(self.model.optimizer.lr)

        # Set the initial learning rate
        K.set_value(self.model.optimizer.lr, start_lr)

        callback = LambdaCallback(on_batch_end=lambda batch,
                                                      logs: self.on_batch_end(batch, logs))

        self.model.fit_generator(generator=generator,
                                 epochs=epochs,
                                 steps_per_epoch=steps_per_epoch,
                                 callbacks=[callback],
                                 **kw_fit)

        # Restore the weights to the state before model fitting
        self.model.load_weights('tmp.h5')

        # Restore the original learning rate
        K.set_value(self.model.optimizer.lr, original_lr)

    def plot_loss(self, n_skip_beginning=10, n_skip_end=5, x_scale='log'):
        """
        Plots the loss.
        Parameters:
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
        """
        plt.ylabel("loss")
        plt.xlabel("learning rate (log scale)")
        plt.plot(self.lrs[n_skip_beginning:-n_skip_end], self.losses[n_skip_beginning:-n_skip_end])
        plt.xscale(x_scale)
        plt.show()

    def plot_loss_change(self, sma=1, n_skip_beginning=10, n_skip_end=5, y_lim=(-0.01, 0.01)):
        """
        Plots rate of change of the loss function.
        Parameters:
            sma - number of batches for simple moving average to smooth out the curve.
            n_skip_beginning - number of batches to skip on the left.
            n_skip_end - number of batches to skip on the right.
            y_lim - limits for the y axis.
        """
        derivatives = self.get_derivatives(sma)[n_skip_beginning:-n_skip_end]
        lrs = self.lrs[n_skip_beginning:-n_skip_end]
        plt.ylabel("rate of loss change")
        plt.xlabel("learning rate (log scale)")
        plt.plot(lrs, derivatives)
        plt.xscale('log')
        plt.ylim(y_lim)
        plt.show()

    def get_derivatives(self, sma):
        assert sma >= 1
        derivatives = [0] * sma
        for i in range(sma, len(self.lrs)):
            derivatives.append((self.losses[i] - self.losses[i - sma]) / sma)
        return derivatives

    def get_best_lr(self, sma, n_skip_beginning=10, n_skip_end=5):
        derivatives = self.get_derivatives(sma)
        best_der_idx = np.argmax(derivatives[n_skip_beginning:-n_skip_end])
        return self.lrs[n_skip_beginning:-n_skip_end][best_der_idx]
